In [1]:
import pandas as pd
import jieba
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.metrics import precision_score, recall_score, f1_score
from scipy.stats import mode
import re

In [2]:
##SWSR dataset
comments = pd.read_csv('SWSR_SexComment.csv')
comments.head()

,index,weibo_id,comment_text,gender,location,like,date,label,category,target
0,0,J1zF7smOR,她们不结婚不要孩子，却觉得我们这种结婚有孩子的是婚驴,女,广东,14,12/05/2020 15:04,1,SCB,G
1,1,GvXf0ymdJ,妈的什么时候有人来性侵我一下啊 不要封口费的那种,男,其他,36,23/08/2018 13:10,1,SO,I
2,2,J2XeR8rPn,我觉得这就是对一类人所有的特点进行形容吧，“爹味”也是贬义词，为什么不觉得有性别歧视意味呢？,女,浙江,0,21/05/2020 16:12,1,MA,G
3,3,IxpfTCMAo,姐姐你不要急，关注久的都知道她骂的一切归根结底都是渴婚的，她并不骂学艺术的，她骂的是不肯像培...,女,江苏,1,31/03/2020 13:57,1,MA,I
4,4,J4B0Mps5Q,"给他们争取权利的不是女权是腐女，你该不会说腐女都女权吧？连女同性恋都不是全部女权，腐女？,男...",女,其他,0,##########,1,MA,G


In [104]:
df = comments.copy()

In [4]:
df["tokens"] = df['comment_text'].astype(str).apply(lambda x: list(jieba.cut(x)))

#training word2vec
w2v_model = Word2Vec(sentences=df["tokens"], vector_size=100, window=5, min_count=2, workers=4, seed=42)

def get_avg_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

df["vector"] = df["tokens"].apply(lambda x: get_avg_vector(x, w2v_model))
comment_vectors = np.vstack(df["vector"].to_numpy())

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/p_/wr6zqx2j1816kr86z8yjxk880000gn/T/jieba.cache
Loading model cost 0.235 seconds.
Prefix dict has been built successfully.


In [5]:
kmeans = KMeans(n_clusters=2, random_state=42)
df["cluster"] = kmeans.fit_predict(comment_vectors)

In [6]:
#evaluating using the pre-labeled data
cluster_map = {}
true_labels = df["label"]

for cluster in df["cluster"].unique():
    majority_label = mode(true_labels[df["cluster"] == cluster], keepdims=True).mode[0]
    cluster_map[cluster] = majority_label
    
df["aligned_pred"] = df["cluster"].map(cluster_map)


precision = precision_score(true_labels, df["aligned_pred"])
recall = recall_score(true_labels, df["aligned_pred"])
f1 = f1_score(true_labels, df["aligned_pred"])

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

Precision: 0.0000
Recall:    0.0000
F1 Score:  0.0000


/opt/anaconda3/envs/nlp310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
df.groupby("cluster")["label"].value_counts(normalize=True)
#this comparison shows that regardless of the cluster, the two categories hold the same weight

cluster  label
0        0        0.649506
         1        0.350494
1        0        0.663525
         1        0.336475
Name: proportion, dtype: float64

In [8]:
pd.set_option('display.max_colwidth', None)
for c in df["cluster"].unique():
    print(f"\nCluster {c} sample:")
    display(df[df["cluster"] == c][["comment_text", "label"]].sample(15, random_state=42))


Cluster 1 sample:


,comment_text,label
7557,那女的撒谎了能怎么样啊？你说这些比划不是还是给他洗吗，她就撒谎了能怎么滴，你特么动手没？打人没？打了就别比比了，老实蹲着得了，还洗尼玛啊！,0
5534,听说木胥也没了？嗯。。反正看成片吧。。希望不要面目全非,0
5192,不是吧氧气们这种微博不应该随手一个举报送他归西和这种掐字眼吃鱼吃很好各种挑刺的微博废啥话呢直接随手举报就完了,0
7841,明显是未成年人权益问题 却变成男女之争 现在都学美国那套竖切阶层了吗？真是上当不轻,0
4209,哈哈哈哈哈哈哈哈哈哈哈哈我吃瓜来了 佩服她那种大妈,0
441,女权想希望死一部分蝻的，非常实际了。,1
2606,"你这种男拳还是少说话吧,我寻思男人讲歪道理骂女人也有几千几万赞呢？就女权值得热搜？",1
8880,你以为ddl没人说啥？还是给华盛顿最保险，谁敢说个不字就是歧视。,0
3178,对啊，到处让极端女权反思的，让极端女权去团结的，有病吗？反思什么？团结什么啊？,0
718,过去的你对现在的你：你！你这是极端女权！,1



Cluster 0 sample:


,comment_text,label
7672,这都75了，还有人说强子被人陷害，44岁哪里还行呢,0
5875,香港青少年霸凌，何止被打30个嘴巴！——点新闻耗时逾月深度调查，今天开播！//,0
689,一开始平权的姐妹还是太善良温和了，对男权让步抱有期待，没有认清蛋糕就这么大，你不去争去抢永远不会多得到蛋糕。,1
6202,并非是大多数人，都喜欢煽风点火，落井下石。反而是大多数人，心如明镜，是非分明。在看过亚仁反击般警醒般的文字后，大多数人更能深刻了解刘亚仁是一位怎样的演员、严弘植是一位怎样的社会人，反到以前对他任何的负面报道和猜测，都显得虚无和苍白了。,0
4620,茶并没有公开说后悔和伍迪合作这种话，只是把片酬捐了，也并没有公开诋毁他，我认为这是挺合理的一种做法吧。,0
44,"大部分时候别人都不喜欢听真话，反而愿意听一些虚伪的东西，现在的社会 呵呵,可以让男人硬起来",1
8012,呵呵，故意的呗，他俩有那么多图，偏偏每次都是这套阿汤正好不修边幅的,0
5407,当年迈克杰克逊也被指娈童，直到他去世后，当事人才站出来澄清是撒谎毁谤，这么做只是为了讹钱。谁知道米兔会不会走向与自己初衷诉求相反的方向呢？而且，……这么快？,0
7860,我看完文章，记住的是“强者要帮助那些连努力的机会的没有的人”，类似于以前有人说过的“幸运的人，有责任去帮助那些没有那么幸运的人”，深以为然。,0
6465,结果？结果啊就是最后消息被打压，然后事情平息，大家淡忘,0


In [9]:
#it seems like there are some ambiguities in the labelling made by the authors, where some sexist posts are misclassified or overlooked. this may have potentially contributed to the lack of precision & accuracy?

#if we were to look at each clusters, there seems to be some commonalities such as cluster 1 has shorter, more harsh toned language, whereas cluster 0 has longer and more thought/policy oriented topics. 

In [10]:
 df['aligned_pred'].unique() #it seems that this pre-liminary method onnly predicts 0 which is non-misogynistic

array([0])

In [11]:
#if the labelling was the issue for evaluation, lets try doing the same thing but for a manually labeled dataset which is shorter but hopefully more consistent.

In [12]:
alt = pd.read_csv("sampled_keyword_df.csv")

In [13]:
alt.head()

,mid,text,userid,username,created_at,reposts_count,comments_count,attitudes_count,keyword,sentiment_score,snownlp_sentiment,misogyny_label
0,4744160006113240,最后一小时。拼手速时间。劵马上🈚️。一小时后涨20块钱。。。王小卤的虎皮鸡爪，多好吃不说了哈哈哈。反正就是一小时后，立涨20块。。<br /><br />爱吃鸡爪的的媎妹可以看看。39块钱两大包，一包里面6-7个鸡爪。🔗扔评论了,1833569404,辛德瑞拉小卖部,2022-03-06 23:09:10+08:00,0,36,118,媎妹,0.549855,neutral,0
1,4867011886449276,极端偏激女权主义分子特征，善良的男性们，你们要警惕啊！,6444840645,好时代见证记录者,2023-02-08 23:18:41+08:00,6,22,49,极端女权,0.975390,positive,1
2,4757606730039874,人们对于不认同的男性发言，带上的标签是“大男子主义”或者是对其个人的剖析，对于不认同的女性发言，带上的标签是“境外势力”“极端女权”“田园女拳”“拳师”<br />有没有官方声讨过那些以侮辱女性为己任的男人，没有。因为他们不是极端男权，而是概括为没有素质或者没有文化。,7514394135,Seok_Cheon上班摸鱼版,2022-04-13 01:41:39+08:00,93,141,1826,田园女拳,0.294325,negative,-1
3,5102628826251561,媎妹感觉你智商为0//<a href='/n/Spidpire'>@Spidpire</a>:说白了就是没钱给工资呗，一年1.2kw，从去年开始压合同价格，压到现在还是这些,1837223697,玟咚Yusi,2024-11-20 03:36:17+08:00,0,0,0,媎妹,0.023232,negative,1
4,5109772649763757,"最近女硕士失踪13年被“收留”大孩子12岁的新闻很火。我根本不想讨论，因为没有意义了：<br />韩女：“我的胯下 不能生出歧视我的性别。我的血肉 不能变成刺向我的尖刀。”<br />我们：“瑞士卷八个我能吃几个”<span class=""url-icon""><img alt=""[老师爱你]"" src=""https://face.t.sinajs.cn/t4/appstyle/expression/ext/normal/82/2023_Teacherlovesyou_org.png"" style=""width:1em; height:1em;"" /></span><br />瑞士卷事件演变到最后变成问男朋友/老公 我能吃几个的时候，我就觉得 问出这句话的女性同 ...<a href=""/status/5109772649763757"">全文</a>",5522145332,小小瑞_171,2024-12-09 20:43:17+08:00,0,0,6,女性权利,0.999536,positive,-1


In [14]:
def clean_text(text):
    if not text:
        return ""
    # Remove hyperlinks and anchor tags
    text = re.sub(r"http\S+|www\S+|<a.*?>|</a>", "", text)
    # Remove all span classes (surl-text, url-icon, and others)
    text = re.sub(r"<span.*?>.*?</span>", "", text)
    # Remove hashtags (content inside ##)
    text = re.sub(r"#.*?#", "", text)
    # Remove any remaining HTML tags
    text = re.sub(r"<.*?>", "", text)
    # Remove emojis and special characters
    text = re.sub(r"[^\w\s\u4e00-\u9fff]", "", text)
    return text.strip()

# Apply the function to the text column
alt["cleaned_text"] = alt["text"].apply(clean_text)

# Display cleaned text
alt[['cleaned_text']].head()

,cleaned_text
0,最后一小时拼手速时间劵马上一小时后涨20块钱王小卤的虎皮鸡爪多好吃不说了哈哈哈反正就是一小时后立涨20块爱吃鸡爪的的媎妹可以看看39块钱两大包一包里面67个鸡爪扔评论了
1,极端偏激女权主义分子特征善良的男性们你们要警惕啊
2,人们对于不认同的男性发言带上的标签是大男子主义或者是对其个人的剖析对于不认同的女性发言带上的标签是境外势力极端女权田园女拳拳师有没有官方声讨过那些以侮辱女性为己任的男人没有因为他们不是极端男权而是概括为没有素质或者没有文化
3,媎妹感觉你智商为0Spidpire说白了就是没钱给工资呗一年12kw从去年开始压合同价格压到现在还是这些
4,最近女硕士失踪13年被收留大孩子12岁的新闻很火我根本不想讨论因为没有意义了韩女我的胯下 不能生出歧视我的性别我的血肉 不能变成刺向我的尖刀我们瑞士卷八个我能吃几个瑞士卷事件演变到最后变成问男朋友老公 我能吃几个的时候我就觉得 问出这句话的女性同 全文


In [40]:
#because we are only doing a binary labelling, i will make the -1 (feminist) into 0s
alt["misogyny_label"] = alt["misogyny_label"].replace(-1, 0) 
alt = alt.dropna()

In [140]:
alt["tokens"] = alt['cleaned_text'].astype(str).apply(lambda x: list(jieba.cut(x)))

w2v_model = Word2Vec(sentences=alt["tokens"], vector_size=100, window=5, min_count=2, workers=4, seed=42)

def get_avg_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

alt["vector"] = alt["tokens"].apply(lambda x: get_avg_vector(x, w2v_model))
comment_vectors = np.vstack(alt["vector"].to_numpy())

kmeans = KMeans(n_clusters=2, random_state=42)
alt["cluster"] = kmeans.fit_predict(comment_vectors)

cluster_map = {}
true_labels = alt["misogyny_label"]

for cluster in alt["cluster"].unique():
    majority_label = mode(true_labels[alt["cluster"] == cluster], keepdims=True).mode[0]
    cluster_map[cluster] = majority_label

alt["aligned_pred"] = alt["cluster"].map(cluster_map)

precision = precision_score(true_labels, alt["aligned_pred"])
recall = recall_score(true_labels, alt["aligned_pred"])
f1 = f1_score(true_labels, alt["aligned_pred"])

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

Precision: 0.0000
Recall:    0.0000
F1 Score:  0.0000


/opt/anaconda3/envs/nlp310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [144]:
alt.groupby("cluster")["misogyny_label"].value_counts(normalize=True) #similar skewness as the previous dataset, which leads me to believe that may the clustering is capturing something? 

cluster  misogyny_label
0        0                 0.654450
         1                 0.345550
1        0                 0.640777
         1                 0.359223
Name: proportion, dtype: float64

In [50]:
pd.set_option('display.max_colwidth', None)
for c in alt["cluster"].unique():
    print(f"\nCluster {c} sample:")
    display(alt[alt["cluster"] == c][["cleaned_text", "misogyny_label"]].sample(15, random_state=42))


Cluster 0 sample:


,cleaned_text,misogyny_label
470,1679吐槽末世重生你摆烂吧我直接开卷作者在第一章作话声称自己爱女但是不厌男会收养一个小孩性别是伏笔我随意翻了一下还没有看到收养小孩的情节但是会这么说十有八九是男孩大概浏览了一下高光男角色可能很多以及第一章有写到男明星坚持华流才是最吊的观念 全文,0
482,先有女性在男性绝对力量前的无力最后追出去的时候我在想为什么还要追再是出来几个男性轮流随地小便男人真恶臭真恶臭真恶臭,0
321,我感觉自从上了大学整个人都不好了鲍婷婷变得也贱总是能用一句话让我烦躁这姐们难道不知道自己说话真的很难听吗而且还老是说自己是个理智粉我倒是没看出来你一遇到你家脑子都被驴踢了一样三观让我炸裂人家老师上课声音小叫男生转述一下你说人家老师魅男咋啥都 全文,1
208,揭秘真相并非针对个人而是厌女与漠视创新之战陈丽君与浙江小百花越剧团遭遇的无情打击背后的深层次矛盾,0
290,三月新推送写得特别好在外网真很多女权博主觉得不用谈彩礼不用谈冠姓权因为和激女无关但正如这篇推送所言我们不该仅仅是男权下的幸存者而应该是男权制度的掘墓人,0
66,不吐不快那个狗屁教授批评讽刺的不是陈丽君一个人而是整个小百花的女性演员这种人当教授 真是讽刺啊在场的人也跟着大笑更是讽刺,0
32,曾几何时窦昭前世于黑暗中独自挣扎家族的辜负如利箭穿心然重生之门开启她誓言抛开一切繁文缛节只为自己而活向那些背叛者讨回公道,0
73,57讨论想讨论下把女权当时尚单品这一现象或者这一说法不太明确具体是怎么定义的我个人目前觉得只要不是打着女权名义做一些损害女性权益的事情我都可以接受,0
394,生于忧患死于安乐堡垒往往是从内部被攻破 全文,1
346,南韩历届总统最适合拍大男主大女主影视作品的在我看来只有朴正熙和全斗焕拍其他人都没什么意思,0



Cluster 1 sample:


,cleaned_text,misogyny_label
456,昨天发了女拳2很惊喜能收到金多美nice 的私信 其实我发现大龄没结婚的男生女生他们不知道急不急反正有一点可以肯定的他们身边的人是挺急的总是旁敲侧击的打听有没有谈对象有没有结婚的打算有人是真关心有人是真八卦时间长了人们就不会当着你的面问了 全文,0
13,激女向前跑的风会擦干我脸上的泪,0
74,原价21658元券后价21058元 21058 么有一个人他的一条短信就可以让你立刻微笑从不曾想过有一天我会变得淡然不再疯狂我有两个我一个冷漠一个热情你怎么对我我就怎么对你就是这样有时候某些谈婚论嫁的剩女还不如谈荤论价的妓女他让你,1
78,真的会有人把大女主 女权这种字眼挂在嘴边吗 女孩我们做的一切都不要贴标签好不好,1
35,20241212 今天我们决定分开了其实这个决定挺难受的不是因为不爱也不是因为有第三者而是我们互相的不理解我们总是希望对方能够给予平等的爱但是发现互相理解真的好难好累坚持不下去了,0
82,我经常看到一些女权读者以不够女权的名义攻击女作者有人问为什么不骂np男频读者常常回答骂了他们也不会改的有时候真的想问女权主义已经到党同伐异分蛋糕的阶段了吗我们难道不应该团结更多可以团结的力量吗又或者你真的是为了女性利益吗你真的如你所说般伟大吗 全文,0
125,城会玩系列母亲节快到了偶遇一场盛大的妈妈出逃计划真的好好她们节日值得这样的庆祝妈妈除了是妈妈也是她自己躺一会逃一阵歇一下从被赋予的身份中脱离出来尽情享受成为自己的时光不仅是节日的度过方式也是她们在生活中无时无刻都应该保有的权利不愧是一直贴心陪 全文,0
283,最新火辣的女性反派角色鞭炮女的魅力真是无人能挡,1
382,我可不是厌男症啊我可不会说出没有男人我会过得更好的那种话找到好男人是很重要的我只是被猪精耽误了找不到好男人而已,1
58,我早说了女q甚至激n内部也会有冲突也会有矛盾相互之间有利益联合才能形成稳固的团结现在背刺这个词语的门槛过低没有过利益共同体何来背刺不能因为别人和自己聊了几句天就觉得是自己人这种事有很多次了只要有观点不同就会换来辱ma又背刺又割席了这不是在胡 全文,0


In [52]:
# it seems that despite there are misclassification occuring because nuances and sarcasm is not captured well, there is still an interesting pattern that distinguishes cluster 1 and 0. In both situations, one cluster is more in length than another, more emotional and more anger (often categorized as hate). So while this cannot be used to pinpoint the shifts of language used for misogyny, it does capture patterns of discussion. 

In [122]:
df["tokens"] = df['comment_text'].astype(str).apply(lambda x: list(jieba.cut(x)))

w2v_model = Word2Vec(sentences=df["tokens"], vector_size=100, window=5, min_count=2, workers=4, seed=42)

def get_avg_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

df["vector"] = df["tokens"].apply(lambda x: get_avg_vector(x, w2v_model))
comment_vectors = np.vstack(df["vector"].to_numpy())

In [124]:
inertias = []
k_range = range(2, 11)  # You can adjust the upper bound

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(comment_vectors)
    inertias.append(kmeans.inertia_)

In [130]:
inertias #seems like 7 clusters is a decent estimate as the change between each intertia is less

[8766.2919921875,
 7698.927734375,
 7098.6611328125,
 6657.513671875,
 6387.72607421875,
 6076.91845703125,
 5832.0712890625,
 5650.4072265625,
 5501.5712890625]

In [132]:
kmeans = KMeans(n_clusters=7, random_state=42)
df["cluster"] = kmeans.fit_predict(comment_vectors)

true_labels = df["label"]
cluster_map = {}
for cluster in df["cluster"].unique():
    majority_label = mode(true_labels[df["cluster"] == cluster], keepdims=True).mode[0]
    cluster_map[cluster] = majority_label

df["aligned_pred"] = df["cluster"].map(cluster_map)

precision = precision_score(true_labels, df["aligned_pred"])
recall = recall_score(true_labels, df["aligned_pred"])
f1 = f1_score(true_labels, df["aligned_pred"])

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

Precision: 0.5407
Recall:    0.1225
F1 Score:  0.1998


In [136]:
df.groupby("cluster")["label"].value_counts(normalize=True)

cluster  label
0        0        0.642069
         1        0.357931
1        0        0.624835
         1        0.375165
2        1        0.540656
         0        0.459344
3        0        0.729801
         1        0.270199
4        0        0.728593
         1        0.271407
5        0        0.593722
         1        0.406278
6        0        0.745000
         1        0.255000
Name: proportion, dtype: float64

In [134]:
pd.set_option('display.max_colwidth', None)
for c in df["cluster"].unique():
    print(f"\nCluster {c} sample:")
    display(df[df["cluster"] == c][["comment_text", "label"]].sample(5, random_state=42))


Cluster 5 sample:


,comment_text,label
1508,"反基是反动吧女权主义那坨日本女人的论调，再说一遍它们是隶属于百度反动吧西方世界的，连日本之家都把它们叫兄弟，懂吗？村狗！中国自己的女权只有女权之声 新媒体女性 吕频等懂事吗,出口闭口中国的女权贴吧？连女权主义吧是一群日本人建的都不知道,呵呵，同胞？你也不是什么好东西！脑子也不行！天天在网上污蔑中国女性的好像就是你们这种公狗吧，你确定是外国人骂中国女的？还是你这种把不到妹的仇女吊丝在天涯编造出世界公交车这种词侮辱国内女性，有一句评论说的好你这种村 比骂中国女人世界公交车，那外国女不是宇宙公交车了",1
3864,"女明星也没有他说的那么肮脏不堪，文章里写了女明星通病巴拉巴拉，直说前任何必要把女明星的道德行为拉下水。他把个人的拿不上台面的思想出书，对同行女明星可有尊重？,女人不老实，男人打了她一顿，让她知道男人的底线，这是对的吗？错的不是她的妻子包揽了所有家务，是他认为女人都应该温良恭谦相夫教子遵守所谓传统美德，这是他个人片面的观念，既然有人愿意挨那也没什么，但是，这种片面观念出书了是对社会思想文化的一种潜意识引导，抨击也没什么不对。",0
1330,我有点不大明白的地方，wsc算上层男性，那上层女性不就是富婆吗？每天背着名牌包包，一天到晚去美容院小日子过的比中层女性滋润多了吧？,1
6866,哈哈，微博有个我关注的人在我关注的人下面评论的话也有提示，,0
1287,昨晚开始我只站平权，就算真的倡导不结婚，也轮不到一个已婚有两个孩子的女权博主骂别人婚驴,1



Cluster 3 sample:


,comment_text,label
4226,确实很难 不然为什么婚姻法修改的越来越完善了呢,0
6641,慕了 我和你的观点一样 但我爹觉得生孩子很好 他讲不生孩子就不会和我麻麻结婚,0
501,一群屌子天天在那说自己睡到了xxx 我她妈 一脚给你踢断,1
996,这样的目的是为了引发国内的社会矛盾 女拳当然有人结工资了 境外势力巴不得多培养些女拳 跟动保公知一个金主,1
6171,静候结果 如果德普说的是真的 我要骂死她,0



Cluster 2 sample:


...

,comment_text,label
8125,章文：同样的事儿，凭什么不同的人不同的待遇？我被全网吊打，zhu军却享受删踮屏蔽的保护，哪儿说理去啊？,0
5412,看到这个消息，真的挺震惊的！当初很喜欢黄手帕里的他，还看过很早的他演的《都市男女》！这种情景，这种结局，不胜唏嘘。,0
5126,云南偏远地区很多12、13岁就同居生小孩了。等到法定年龄再去打结婚证。打结婚证的时候，小孩都好几个了。,0
911,我发现你跟果子狸、写字楼大妈那些无脑女权相比更加理智平和（抱歉把你跟他们放在一起，有辱身份）,1
6789,感觉真的是。。pgone这次真的表现很糟糕了以及vava这次真的很棒按发挥来讲妥妥vava啊，不过如果刷了pg1万粉可能不买账了毕竟是节目算了不要太认真,0



Cluster 4 sample:


,comment_text,label
7273,这人贴标签倒挺溜。怎么会关注你啊…三观严重不和啊，……,0
7458,反正我是还没结婚的单身狗，我觉得这并没有什么。拿行李也可能谁比较顺手谁就拿了而已。难道还因为没帮忙拿行李跟男朋友分手？拿不了不会叫他帮忙吗。我觉得这才是夫妻真实的相处模式吧，哪来那么多那种霸道总裁情节，天天宠你舍不得你拿一点点重的东西。那在家做个饭是不是还得说朗朗居然叫老婆做饭,0
5477,"哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈""你不结婚?全像你这样了，我怎么办。你更欠我的了"",所以啊，很明显婚姻是他们最后坚持的底线了，痛死他们算了",0
5692,就爆料了一个事情，是真是假都不知道，就让我们猜…………,0
5627,什么玩意儿我看你是生活中太渣没有责任感没有情商没有女朋友跑到这辱骂别人,0



Cluster 0 sample:


,comment_text,label
3180,揪个虫，girlboss这个词本身其实是有争议的，在英文含义里有小妞老板的意思，有国外的女权小伙伴曾经指出过这个词不妥。我因为之前有相关的工作需要，所以查过这个词,0
4204,广大因政治正确被歧视被边缘化被噤声的劳动阶层以及很多真正为那些民主党不着边际只为迎合政治正确制定的政策埋单的被压榨的中产。,0
273,哪怕社会地位是女性为强势也会出现这样的问题，因为与性别地位孰强孰弱无关，与人本身有关,1
8496,看的我情绪非常低落的一本书。感慨不是作为普通人的诗织被有权势的山口性侵的事件本身。而是当事件发生后，社会各系统对于性侵案件的处理方式和态度，让被害人遭受心理上的多次“性侵害”。或许诗织的工作身份让她有更坚定的信念去发掘真相，可这需要顶住多大的压力才能一步步走下去，读的我胆战心惊。,0
910,哗众取宠！第一，打人行为是错的，这个无论性别信仰和立场。第二，为何要刻意强调性别？强调是男人打女人却获得了其他女人的维护，而忽略了立场和信念，是女权主义者打了反女权主义者的工作者而后被女权主义者维护，两者表述差别太大，混淆视听。第三，被打者服务报社做了什么？,1
